In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df=pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.optim import Adam

In [6]:
#df.drop(['CD Account'],inplace=True,axis=1)

In [7]:
#df.drop(['Securities Account'],axis=1,inplace=True)

In [8]:
df["Experience"]=df["Experience"].apply(lambda x: x if x>0 else 0)

In [9]:
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [10]:
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [11]:
columns=['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Online', 'CreditCard','Personal Loan']
df=df[columns]

In [12]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.30,random_state=69)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(xtrain)
x_test=sc.transform(xtest)

In [15]:
x_train=torch.from_numpy(x_train).to(torch.float32)
y_train=torch.from_numpy(ytrain).to(torch.float32)

In [16]:
bankdata=TensorDataset(x_train,y_train)
batch_size=16
data = DataLoader(bankdata, batch_size=batch_size, shuffle=True)

In [17]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.hidden=nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid=nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.hidden(x)
        x = self.layer2(x)
        x = self.sigmoid(x) 
        return x

In [18]:
input_size = x_train.shape[1]
hidden_size = 32
output_size = 1
mlp = Model(input_size, hidden_size, output_size)
mlp

Model(
  (layer1): Linear(in_features=11, out_features=32, bias=True)
  (hidden): ReLU()
  (layer2): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [19]:
loss_func = nn.BCELoss()
optimizer = Adam(mlp.parameters(), lr=0.01)

In [20]:
epoch = 20
training_loss = [0] * epoch
training_accuracy = [0] * epoch

for i in range(epoch):
    epochloss=0
    acc=0
    for x_batch, y_batch in data:
        pred_y=mlp(x_batch)
        loss=loss_func(pred_y,y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epochloss+=loss.item()*x_batch.size(0)
        cnt=(torch.where(pred_y>=0.5,1,0)==y_batch.unsqueeze(1)).sum().float().item()
        acc+=cnt
    print(i,epochloss/len(data.dataset),acc/len(data.dataset))
    
        

0 0.15716732813205037 0.9462857142857143
1 0.08749086732204471 0.9685714285714285
2 0.07799919943245394 0.9734285714285714
3 0.07149363011441061 0.9751428571428571
4 0.06512898830191366 0.9774285714285714
5 0.06373648367276681 0.9765714285714285
6 0.06174191977402994 0.9791428571428571
7 0.05895527335362775 0.9817142857142858
8 0.05611988651672644 0.9802857142857143
9 0.05506201872329361 0.9825714285714285
10 0.053692905464874846 0.9828571428571429
11 0.0511508636564166 0.9805714285714285
12 0.05267668656425251 0.9814285714285714
13 0.04928433113651616 0.984
14 0.04598175855900627 0.9842857142857143
15 0.04996899534595598 0.9837142857142858
16 0.04641675842643183 0.9828571428571429
17 0.04700174824078567 0.9848571428571429
18 0.04348428814685238 0.9848571428571429
19 0.04565850577960789 0.9868571428571429


In [21]:
x_test=torch.from_numpy(x_test).to(torch.float32)
y_test=torch.from_numpy(ytest).to(torch.float32)

In [30]:
bankdata=TensorDataset(x_test,y_test)
batch_size=len(x_test)
data = DataLoader(bankdata, batch_size=batch_size, shuffle=True)

In [31]:
for x,y in data:
    pred=mlp(x)
    y=y.reshape(-1,1)
    cnt=(torch.where(pred>=0.5,1,0)==y).sum().float().item()
print(cnt/len(data.dataset))

0.976


In [36]:
def fit(mlp):
    pred_y=mlp(x_train)
    accuracy =(torch.where(pred_y>=0.5,1,0).flatten()==y_train).sum().float().item() / 3500
    return accuracy

In [37]:
def cross(parent1,parent2):
    
    shape = [i.numpy().shape for i in parent1.parameters()]
    size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

    pt1=np.concatenate([i.numpy().flatten() for i in parent1.parameters()])
    pt2=np.concatenate([i.numpy().flatten() for i in parent2.parameters()])
    e=len(pt2)
    point=random.randrange(0,e)
    child1=np.concatenate([pt1[:point], pt2[point:]])
    child2=np.concatenate([pt2[:point], pt1[point:]])
    
    
    l1=[]
    l1=random.sample(range(0,len(child1)),2)
    mutchild1=child1.copy()
    temp=child1[l1[1]]
    mutchild1[l1[1]]=child1[l1[0]]
    mutchild1[l1[0]]=temp
    
    l2=[]
    l2=random.sample(range(0,len(child1)),2)
    mutchild2=child2.copy()
    tem=child2[l2[1]]
    mutchild2[l2[1]]=child2[l2[0]]
    mutchild2[l2[0]]=temp
    
    children=[child1, child2, mutchild1, mutchild2]
    out=list()
    for ch in children:
        pa=list()
        c_sum=0
        for i in range(len(size)):
            array = ch[c_sum : c_sum + size[i]]
            array = array.reshape(shape[i])
            c_sum += size[i]
            pa.append(array)
        pa = np.array(pa, dtype="object")
        out.append(pa)
    
    out = np.array(out, dtype="object")
    return out 

In [45]:
torch.manual_seed(420)
torch.set_grad_enabled(False)
pop_size=10
input_size = x_train.shape[1]
hidden_size = 4
output_size = 1
def train(n):
    pop=np.array([Model(input_size, hidden_size, output_size) for i in range(pop_size)])
    best=0
    for i in range(n):    
        function=[]
        function+=[fit(spop) for spop in pop]
        temp=np.argsort(function)
        pop=np.array(pop)[temp]
        best=pop[-1]
        p1=pop[-1]
        p2=pop[2]
        p3=pop[-2]
        p4=pop[3]
        c1=cross(p1,p2)
        c2=cross(p3,p4)
        c=np.concatenate([c1, c2])
        new_pop = np.array([Model(input_size, hidden_size, output_size) for i in range(len(c))])
        for count, ml in enumerate(new_pop, 0):
            for index, pa in enumerate(ml.parameters(), 0):
                pa.data = (torch.tensor(c[count][index]))
        pop = new_pop.copy()
    return best

In [46]:
bestmodel=train(10)

In [48]:
from sklearn.metrics import classification_report
pred_y = bestmodel(x_test)
pred_y = torch.where(pred_y>=0.5, 1, 0).flatten()
print(classification_report(pred_y,y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95      1500
         1.0       0.00      0.00      0.00         0

    accuracy                           0.90      1500
   macro avg       0.50      0.45      0.47      1500
weighted avg       1.00      0.90      0.95      1500



E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
